In [ ]:
! pip install --upgrade --quiet google-genai

In [2]:
import os

PROJECT_ID = "qwiklabs-gcp-01-4cb6791a06aa"
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [5]:
os.environ.get("GOOGLE_CLOUD_PROJECT")

'qwiklabs-gcp-01-4cb6791a06aa'

In [4]:
os.environ.get("GOOGLE_CLOUD_REGION")

'us-central1'

In [3]:
PROJECT_ID, LOCATION

('qwiklabs-gcp-01-4cb6791a06aa', 'us-central1')

In [6]:
client

In [7]:
from IPython.display import Markdown, display
from google.genai.types import FunctionDeclaration, GenerateContentConfig, Part, Tool
import requests

In [9]:
MODEL_ID = "gemini-2.0-flash-001"

def get_location(
    amenity: str | None = None,
    street: str | None = None,
    city: str | None = None,
    county: str | None = None,
    state: str | None = None,
    country: str | None = None,
    postalcode: str | None = None,
) -> list[dict]:
    """
    Get latitude and longitude for a given location.

    Args:
        amenity (str | None): Amenity or Point of interest.
        street (str | None): Street name.
        city (str | None): City name.
        county (str | None): County name.
        state (str | None): State name.
        country (str | None): Country name.
        postalcode (str | None): Postal code.

    Returns:
        list[dict]: A list of dictionaries with the latitude and longitude of the given location.
                    Returns an empty list if the location cannot be determined.
    """
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "amenity": amenity,
        "street": street,
        "city": city,
        "county": county,
        "state": state,
        "country": country,
        "postalcode": postalcode,
        "format": "json",
    }
    # Filter out None values from parameters
    params = {k: v for k, v in params.items() if v is not None}

    try:
        response = requests.get(base_url, params=params, headers={"User-Agent": "none"})
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching location data: {e}")
        return []

In [10]:
prompt = """
I want to get the coordinates for the following address:
1600 Amphitheatre Pkwy, Mountain View, CA 94043
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=[get_location], temperature=0),
)
print(response.text)

The coordinates for 1600 Amphitheatre Pkwy, Mountain View, CA 94043 are: latitude 37.4224857, longitude -122.0855846.



In [13]:
prompt = """
I want to get the coordinates for the following address:
Bat Yam Promenade, Bat Yam, Israel, Postal Code 59100
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=[get_location], temperature=0),
)
print(response.text)

I was not able to find the coordinates for the provided address. Would you like me to try a different address or modify the current one?



In [28]:
prompt = """
I want to get the coordinates for the following address:
Bat Yam Promenade, Israel
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=[get_location], temperature=0),
)
print(response.text)

I was not able to find the coordinates for the given address. I can try again if you provide me with more information, like city or postal code.



In [31]:
prompt = """
I want to get the coordinates for the following address:
Bat Yam Beach, Israel
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=[get_location], temperature=0),
)
print(response.text)

I was not able to find the coordinates for the location you provided. I can try again if you provide me with more information. For example, I could use the city, county, or postal code to narrow down the search.



In [26]:
prompt = """
I want to get the coordinates for the following address:
Bat Yam 59100, Israel
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=[get_location], temperature=0),
)
print(response.text)

The coordinates for Bat Yam, Israel are:
Latitude: 32.0154565
Longitude: 34.7505283



In [27]:
response

GenerateContentResponse(
  automatic_function_calling_history=[
    UserContent(
      parts=[
        Part(
          text="""
I want to get the coordinates for the following address:
Bat Yam 59100, Israel
"""
        ),
      ],
      role='user'
    ),
    Content(
      parts=[
        Part(
          function_call=FunctionCall(
            args={<... 3 items at Max depth ...>},
            name='get_location'
          )
        ),
      ],
      role='model'
    ),
    Content(
      parts=[
        Part(
          function_response=FunctionResponse(
            name='get_location',
            response={<... 1 item at Max depth ...>}
          )
        ),
      ],
      role='user'
    ),
  ],
  candidates=[
    Candidate(
      avg_logprobs=-0.013856232166290283,
      content=Content(
        parts=[
          Part(
            text="""The coordinates for Bat Yam, Israel are:
Latitude: 32.0154565
Longitude: 34.7505283
"""
          ),
        ],
        role='model'
      ),


In [15]:
# chat
get_product_info = FunctionDeclaration(
    name="get_product_info",
    description="Get the stock amount and identifier for a given product",
    parameters={
        "type": "OBJECT",
        "properties": {
            "product_name": {"type": "STRING", "description": "Product name"}
        },
    },
)

get_store_location = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
        "type": "OBJECT",
        "properties": {"location": {"type": "STRING", "description": "Location"}},
    },
)

place_order = FunctionDeclaration(
    name="place_order",
    description="Place an order",
    parameters={
        "type": "OBJECT",
        "properties": {
            "product": {"type": "STRING", "description": "Product name"},
            "address": {"type": "STRING", "description": "Shipping address"},
        },
    },
)

retail_tool = Tool(
    function_declarations=[
        get_product_info,
        get_store_location,
        place_order,
    ],
)

In [16]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        temperature=0,
        tools=[retail_tool],
    ),
)

In [17]:
prompt = """
Do you have the Pixel 9 in stock?
"""

response = chat.send_message(prompt)
response.function_calls[0]

FunctionCall(
  args={
    'product_name': 'Pixel 9'
  },
  name='get_product_info'
)

In [18]:
# simulate to the LLM that the fn call
# returns an item from the inventory
api_response = {"sku": "GA04834-US", "in_stock": "yes"}

In [19]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_info",
        response={
            "content": api_response,
        },
    ),
)
display(Markdown(response.text))

Yes, we have the Pixel 9 in stock. Its SKU is GA04834-US.


In [20]:
prompt = """
What about the Pixel 9 Pro XL? Is there a store in
Mountain View, CA that I can visit to try one out?
"""

response = chat.send_message(prompt)
response.function_calls

[FunctionCall(
   args={
     'product_name': 'Pixel 9 Pro XL'
   },
   name='get_product_info'
 ),
 FunctionCall(
   args={
     'location': 'Mountain View, CA'
   },
   name='get_store_location'
 )]

In [21]:
product_info_api_response = {"sku": "GA08475-US", "in_stock": "yes"}
store_location_api_response = {
    "store": "2000 N Shoreline Blvd, Mountain View, CA 94043, US"
}

In [22]:
response = chat.send_message(
    [
        Part.from_function_response(
            name="get_product_info",
            response={
                "content": product_info_api_response,
            },
        ),
        Part.from_function_response(
            name="get_store_location",
            response={
                "content": store_location_api_response,
            },
        ),
    ]
)
display(Markdown(response.text))

Yes, the Pixel 9 Pro XL is in stock with SKU GA08475-US. The store location in Mountain View, CA is 2000 N Shoreline Blvd.


In [23]:
prompt = """
I'd like to order a Pixel 9 Pro XL and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.function_calls

[FunctionCall(
   args={
     'address': '1155 Borregas Ave, Sunnyvale, CA 94089',
     'product': 'Pixel 9 Pro XL'
   },
   name='place_order'
 )]

In [24]:
order_api_response = {
    "payment_status": "paid",
    "order_number": 12345,
    "est_arrival": "2 days",
}

In [25]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content": order_api_response,
        },
    ),
)
display(Markdown(response.text))

OK. I have placed an order for a Pixel 9 Pro XL to be shipped to 1155 Borregas Ave, Sunnyvale, CA 94089. The order number is 12345 and it should arrive in 2 days. The payment status is paid.
